# Renewal Models

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, lognorm


def bpt_pdf(t, mu, alpha):
    """
    Implements the Brownian Passage Time probability density function
    """
    coeff = np.sqrt(mu / (2. * np.pi * (alpha ** 2.) * (t ** 3.)))
    return coeff * np.exp((-(t - mu) ** 2.) / (2. * mu * (alpha ** 2.) * t))

def bpt_cdf(t, mu, alpha):
    """
    Implements the Brownian Passage Time cumulative density function
    """
    u1_t = (1. / alpha) * ((t ** 0.5) * (mu ** -0.5) - (t ** -0.5) * (mu ** 0.5))
    u2_t = (1. / alpha) * ((t ** 0.5) * (mu ** -0.5) + (t ** -0.5) * (mu ** 0.5))
    return norm.cdf(u1_t) + np.exp(2. / (alpha ** 2.)) * norm.cdf(-u2_t)

to_moment = lambda m: 10.0 ** (1.5 * m + 9.05)
to_magnitude = lambda m: (2. / 3.) * (np.log10(m) - 9.05)

Earthquakes are given to have occurred in:

170, 450, 630, 810, 970, 1095, 1310, 1470, 1630, 1760, 1868

Determine the inter-event times:

In [ ]:
event_years = np.array([170., 450., 630., 810., 970., 1095., 1310., 1470., 1630., 1760., 1868.])
inter_event_times = np.diff(event_years)
print(inter_event_times)

Calculate Mean, Standard Deviation and Coefficient of Variation:

In [ ]:
mu = np.mean(np.abs(inter_event_times))
print("Mean inter-event time: %.2f" % mu)
lamda = 1. / mu
print("Average Rate of Events: %.6f" % lamda)
sigma = np.std(np.abs(inter_event_times))
print("Variability in inter-event time %.2f" % sigma)
alpha = sigma / mu
print("Coefficient of variation (aperiodicity) %.4f" % alpha)

Set start date and event date:

In [ ]:
start_date = 2018.
end_date = 2038.
last_event_date = 1868.
d_t = end_date - start_date

### Poisson Model

In a Poisson Model the probability of an event occurring in time, $\Delta T$, years is:

$P \left( {\Delta T | \lambda} \right) = 1 - e^{-\lambda \Delta T}$

where the rate ($\lambda$) = $\frac{1}{\mu}$

In [ ]:
poisson_prob = 1.0 - np.exp(-lamda * d_t)
print("The probability of occurrence in the next %g years\n (according to the Poisson Model) is %.6f" % (d_t, poisson_prob))

### Lognormal Model

In the lognormal model the probability of an event occuring in time $\Delta T$ years is:

$P \left( {t \leq T \leq T + \Delta T | t} \right) = \frac{\int_T^{T + \Delta T} f \left( {t} \right) dt}{\int_T^{\infty} f \left( {t} \right) dt} = \frac{F \left( {T + \Delta T} \right) - F \left( T \right)}{1 - F\left( T \right)}$

where

$f\left( t \right) = \frac{1}{t\sigma_{\ln T}\sqrt(2\pi)} \exp \left( {\frac{-\left( {\ln t - \mu} \right)^2}{2\sigma_{\ln T}^2 }}\right)$

See here for details on the use of scipy.stats.lognorm:

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.lognorm.html

In [ ]:
sigma_lnT = np.std(np.log(inter_event_times))
mu = np.exp(np.mean(np.log(inter_event_times)))

F_T = lognorm.cdf(start_date - last_event_date, s=sigma_lnT, scale=mu)

F_T_DT = lognorm.cdf(end_date - last_event_date, s=sigma_lnT, scale=mu)

lognorm_prob = (F_T_DT - F_T) / (1.0 - F_T)

print("The probability of occurrence in the next %g years\n (according to the Lognormal Model) is %.6f" % (d_t, lognorm_prob))

### Brownian Passage Time (BPT) Model

In the BPT model the probability of an event occuring in time $\Delta T$ years is:

$P \left( {t \leq T \leq T + \Delta T | t} \right) = \frac{\int_T^{T + \Delta T} f \left( {t} \right) dt}{\int_T^{\infty} f \left( {t} \right) dt} = \frac{F \left( {T + \Delta T} \right) - F \left( T \right)}{1 - F\left( T \right)}$

where

$f\left( t \right) = \sqrt{\frac{\mu}{2\pi\alpha^2t^3}} \exp \left( {\frac{-\left( {t - \mu} \right)^2}{2\mu\alpha^2t}}\right)$

In [ ]:
F_T = bpt_cdf(start_date - last_event_date, mu, alpha)
F_T_DT = bpt_cdf(end_date - last_event_date, mu, alpha)

bpt_prob = (F_T_DT - F_T) / (1.0 - F_T)

print("The probability of occurrence in the next %g years\n (according to the BPT Model) is %.6f" % (d_t, bpt_prob))

### Bonus Challenge!

The last event occurred in 1868. Produce a plot showing the probability of occurrence in each year between 1868 and 2050 for each of the three different models: Poisson, Lognormal, Brownian Passage Time